In [1]:
from fastai.collab import *
from fastai.tabular.all import *

In [2]:
path = untar_data(URLs.ML_100k)

In [3]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])

In [4]:
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [22]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=32)
dls.show_batch()

,user,title,rating
0,747,Cinderella (1950),5
1,313,"Devil's Own, The (1997)",3
2,514,Tombstone (1993),3
3,894,Donnie Brasco (1997),4
4,586,Raiders of the Lost Ark (1981),4
5,327,Breakfast at Tiffany's (1961),4
6,234,Lawrence of Arabia (1962),5
7,43,Sabrina (1954),4
8,561,"Magnificent Seven, The (1954)",3
9,95,Snow White and the Seven Dwarfs (1937),4


In [8]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

In [9]:
user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

You can access an element, through vector multiplication

In [10]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([ 1.1948,  1.7080, -1.3444, -0.3565, -0.1517])

In [11]:
user_factors[3]

tensor([ 1.1948,  1.7080, -1.3444, -0.3565, -0.1517])

In [12]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)

    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return (users * movies).sum(dim=1)

In [13]:
x, y =dls.one_batch()

In [17]:
x.shape

torch.Size([64, 2])

In [25]:
model = DotProduct(n_users, n_movies, 10)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [26]:
learn.fit_one_cycle(5, 5e-3)

RuntimeError: Exception occured in `TrainEvalCallback` when calling event `before_fit`:
	CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
